# Model


In [47]:
import glob
import os
import json
from datetime import datetime
import math
import requests

### Obtain today's sunrise/sunset time


In [48]:
LAT = 31.09892
LONG = 121.280426
# url = f"https://api.sunrisesunset.io/json?lat={LAT}&lng={LONG}"
# response = requests.get(url)
# response = response.json()
# sunrise = datetime.strptime(
#     response["results"]["date"] + " " + response["results"]["sunrise"],
#     "%Y-%m-%d %I:%M:%S %p",
# )
# sunset = datetime.strptime(
#     response["results"]["date"] + " " + response["results"]["sunset"],
#     "%Y-%m-%d %I:%M:%S %p",
# )

### Calculate solar incident angle


In [49]:
class cos_solar_incident_angle:
    def __init__(self, time):
        self.tilt_angle = math.radians(5)  # angle from horizontal
        self.azimuth_angle = math.radians(180)  # angle from north
        self.time = datetime.fromtimestamp(time)

        # calculate solar time:
        self.set_day_of_year()
        self.set_equation_of_time()
        self.set_time_correction_factor()
        self.set_ast_corrected_local_solar_time()

        # use solar time to calculate solar angle:
        self.set_hour_angle()  # radians
        self.set_solar_declination()  # radians

        self.set_cos_incident_angle()

    def set_day_of_year(self):
        self.day_of_year = datetime.now().timetuple().tm_yday
        # print(f"day of year: {self.day_of_year}")

    def set_equation_of_time(self):
        B = 360 * (self.day_of_year - 81) / 365
        self.equation_of_time = (
            9.87 * math.sin(math.radians(2 * B))
            - 7.53 * math.cos(math.radians(B))
            - 1.5 * math.sin(math.radians(B))
        )  # minutes
        # print(f"equation of time: {self.equation_of_time} (minutes)")

    def set_time_correction_factor(self):  # 8 refers to UTC+8
        self.time_correction_factor = (
            4 * (LONG - 8 * 15) + self.equation_of_time
        )
        # print(
        #     f"time correction factor: {self.time_correction_factor} (minutes)"
        # )

    def set_ast_corrected_local_solar_time(self):
        self.ast_corrected_local_solar_time = (
            self.time.hour + self.time.minute / 60
        ) - self.time_correction_factor / 60  # hours
        # print(
        #     f"ast corrected local solar time: {self.ast_corrected_local_solar_time} (hours out of 24-hour clock)"
        # )

    def set_hour_angle(self):
        self.hour_angle = (math.pi / 12) * (
            self.ast_corrected_local_solar_time - 12
        )
        # print(f"hour angle: {self.hour_angle} (radians negative)")

    def set_solar_declination(self):
        self.solar_declination = -0.40928 * math.cos(
            (self.day_of_year + 10) * (2 * math.pi / 365)
        )
        # print(
        #     f"solar declination: {self.solar_declination} (radians positive)"
        # )

    def set_cos_incident_angle(self):
        delta = self.solar_declination
        beta = self.tilt_angle
        zeta = self.azimuth_angle
        omega = self.hour_angle
        phi = math.radians(LAT)

        self.cos_incident_angle = (
            (math.sin(delta) * math.sin(phi) * math.cos(beta))
            + (
                math.sin(delta)
                * math.cos(phi)
                * math.sin(beta)
                * math.cos(zeta)
            )
            + (
                math.cos(delta)
                * math.cos(phi)
                * math.cos(omega)
                * math.cos(beta)
            )
            - (
                math.cos(delta)
                * math.sin(phi)
                * math.cos(omega)
                * math.sin(beta)
                * math.cos(zeta)
            )
            - (
                math.cos(delta)
                * math.sin(omega)
                * math.sin(beta)
                * math.sin(zeta)
            )
        )


# angle = cos_solar_incident_angle()
# print(angle.cos_incident_angle)
# print(math.degrees(math.acos(angle.cos_incident_angle)))

### Calculate average incident angle for given hour and step size


In [50]:
def calculate_hourly_average_cos_incident_angle(offset_hour=0, step_size=5):
    """
    Calculate the average cosine of the solar incident angle over the next hour
    Offset hour is the number of hours from the current hour to start calculating the average
    (e.g. offset_hour=0 means start calculating from the current hour, offset_hour=1 means start calculating from the next hour)
    Step size is the number of minutes between each calculation.
    Step size must be a factor of 60.
    """
    if step_size <= 0:
        raise ValueError("step size must be greater than 0")
    elif step_size > 60:
        raise ValueError("step size must be less than 60")
    elif offset_hour < 0:
        raise ValueError("offset hour must be greater than or equal to 0")
    elif offset_hour != int(offset_hour):
        raise ValueError("offset hour must be an integer")
    elif step_size != int(step_size):
        raise ValueError("step size must be an integer")
    elif 60 % step_size != 0:
        raise ValueError("step size must be a factor of 60")

    total_incident_angle = 0
    total_steps = 60 // step_size
    current_hour = (
        datetime.now().timestamp() - datetime.now().timestamp() % 3600
    )
    starting_hour = current_hour + offset_hour * 3600
    for i in range(total_steps):
        current_solar = cos_solar_incident_angle(
            starting_hour + i * step_size * 60
        )
        total_incident_angle += current_solar.cos_incident_angle

    return total_incident_angle / total_steps

### Calculate scaling factor for the given hour


In [51]:
def calculate_power_efficiency_forecast(offset_hour=0, step_size=5):
    """
    Calculate the forecasted solar power output for the next hour
    Offset hour is the number of hours from the current hour to start calculating the average
    (e.g. offset_hour=0 means start calculating from the current hour, offset_hour=1 means start calculating from the next hour)
    """
    list_of_files = glob.glob("data/*")
    latest_file = max(list_of_files, key=os.path.getctime)

    with open(latest_file, "r") as f:
        data = f.read()
        data = json.loads(data)

    # get clouds data
    target_hour = datetime.now().hour + offset_hour
    low_clouds = data["data_1h"]["lowclouds"][target_hour]
    mid_clouds = data["data_1h"]["midclouds"][target_hour]
    high_clouds = data["data_1h"]["highclouds"][target_hour]

    # calculate scaling factor
    C = 0.46
    D = 0.34
    E = 0.08
    F = 0.10

    scaling_factor = (
        C * (100 - low_clouds) / 100
        + D * (100 - mid_clouds) / 100
        + E * (100 - high_clouds) / 100
        + F
    )

    # calculate average cosine of the solar incident angle
    average_cos_incident_angle = calculate_hourly_average_cos_incident_angle(
        offset_hour, step_size
    )

    # calculate forecasted power efficiency
    p_efficiency = average_cos_incident_angle * scaling_factor

    return p_efficiency

In [52]:
print(calculate_power_efficiency_forecast(1, 1))

0.3697856300564688
